In [29]:
from tableauscraper import TableauScraper as TS

url= "https://public.tableau.com/views/moph_covid_v3/Story1"
ts = TS()
ts.loads(url)
workbook = ts.getWorkbook()

for i, t in enumerate(workbook.worksheets):
    print(f"{i} worksheet name : {t.name}") #show worksheet name
    # print(t.data) #show dataframe for this worksheet

0 worksheet name : ppe_available
1 worksheet name : respirator_available
2 worksheet name : map_total
3 worksheet name : doughtnut_total
4 worksheet name : moph_bed
5 worksheet name : N95_available
6 worksheet name : Mask_available
7 worksheet name : Glove_Nitrile_all
8 worksheet name : bed_total
9 worksheet name : province_total
10 worksheet name : niv_available


In [27]:
workbook.worksheets[9].data

,Prov Name-value,Prov Name-alias,Measure Names-alias,Measure Values-value,Measure Values-alias
0,สิงห์บุรี,สิงห์บุรี,เตียงใช้ไปแล้ว,11,11
1,เชียงใหม่,เชียงใหม่,เตียงใช้ไปแล้ว,1,1
2,นครปฐม,นครปฐม,เตียงใช้ไปแล้ว,40,40
3,ลำพูน,ลำพูน,เตียงใช้ไปแล้ว,36,36
4,ตรัง,ตรัง,เตียงใช้ไปแล้ว,38,38
...,...,...,...,...,...
149,นราธิวาส,นราธิวาส,เตียงพร้อมใช้,546,546
150,อุดรธานี,อุดรธานี,เตียงพร้อมใช้,1106,"1,106"
151,สุรินทร์,สุรินทร์,เตียงพร้อมใช้,1483,"1,483"
152,กรุงเทพมหานคร,กรุงเทพมหานคร,เตียงพร้อมใช้,1528,"1,528"


In [30]:
workbook.worksheets[3].data

,AGG(min(1))-value,AGG(min(1))-alias,Measure Names-alias,Measure Values-alias,AGG(bed_percentage)-alias
0,1,1,เตียงพร้อมใช้,"25,630",68.582591
1,1,1,เตียงใช้ไปแล้ว,"11,741",0


In [9]:
def workbook_explore(workbook):
    print()
    print("storypoints: {}", workbook.getStoryPoints())
    print("parameters {}", workbook.getParameters())
    for t in workbook.worksheets:
        print()
        print("worksheet name : {}", t.name)  # show worksheet name
        print(t.data)  # show dataframe for this worksheet
        print("filters: ")
        for f in t.getFilters():
            print("  {} : {} {}", f['column'], f['values'][:10], '...' if len(f['values']) > 10 else '')
        print("selectableItems: ")
        for f in t.getSelectableItems():
            print("  {} : {} {}", f['column'], f['values'][:10], '...' if len(f['values']) > 10 else '')

In [16]:
from loguru import logger
import pandas as pd

In [17]:
def workbook_flatten(wb, date=None, **mappings):
    """return a single DataFrame from a workbook flattened according to mappings
    mappings is worksheetname=columns
    if columns is type str puts a single value into column
    if columns is type dict will map worksheet columns to defined dataframe columns
    if those column names are in turn dicts then the worksheet will be pivoted and the values mapped to columns
    e.g.
    worksheet1="Address",
    worksheet2=dict(ws_phone="phone", ws_state="State"),
    worksheet3=dict(ws_state=dict(NSW="State: New South Wales", ...))
    """
    # TODO: generalise what to index by and default value for index
    res = pd.DataFrame()
    data = dict()
    if date is not None:
        data["Date"] = [date]
    for name, col in mappings.items():
        try:
            df = wb.getWorksheet(name).data
        except (KeyError, TypeError, AttributeError):
            # TODO: handle error getting wb properly earlier
            logger.info("Error getting tableau {}/{} {}", name, col, date)
            continue

        if type(col) != str:
            if df.empty:
                logger.info("Error getting tableau {}/{} {}", name, col, date)
                continue
            # if it's not a single value can pass in mapping of cols
            df = df[col.keys()].rename(columns={k: v for k, v in col.items() if type(v) == str})
            df['Date'] = pd.to_datetime(df['Date']).dt.normalize()
            # if one mapping is dict then do pivot
            pivot = [(k, v) for k, v in col.items() if type(v) != str]
            if pivot:
                pivot_cols, pivot_mapping = pivot[0]  # can only have one
                # Any other mapped cols are what are the values of the pivot
                df = df.pivot(index="Date", columns=pivot_cols)
                df = df.drop(columns=[c for c in df.columns if not any_in(c, *pivot_mapping.keys())])  # Only keep cols we want
                df = df.rename(columns=pivot_mapping)
                df.columns = df.columns.map(' '.join)
                df = df.reset_index()
            df = df.set_index("Date")
            # This seems to be 0 in these graphs. and if we don't then any bad previous values won't get corrected. TODO: param depeden
            df = df.replace("%null%", 0)
            # Important we turn all the other data to numberic. Otherwise object causes div by zero errors
            df = df.apply(pd.to_numeric, errors='coerce', axis=1)

            # Some series have gaps where its assumed missing values are 0. Like deaths
            # TODO: we don't know how far back to look? Currently 30days for tests and 60 for others?
            #start = date - datetime.timedelta(days=10) if date is not None else df.index.min()
            #start = min([start, df.index.min()])
            start = df.index.min()
            # Some data like tests can be a 2 days late
            # TODO: Should be able to do better than fixed offset?
            #end = date - datetime.timedelta(days=5) if date is not None else df.index.max()
            #end = max([end, df.index.max()])
            end = df.index.max()
            assert date is None or end <= date
            all_days = pd.date_range(start, end, name="Date", normalize=True, closed=None)
            try:
                df = df.reindex(all_days, fill_value=0.0)
            except ValueError:
                return pd.DataFrame()  # Sometimes there are duplicate dates. if so best abort the whole workbook since something is wrong

            res = res.combine_first(df)
        elif df.empty:
            # TODO: Seems to mean that this is 0? Should be confirgurable?
            data[col] = [0.0]
        elif col == "Date":
            data[col] = [pd.to_datetime(list(df.loc[0])[0], dayfirst=False)]
        else:
            data[col] = list(df.loc[0])
            if data[col] == ["%null%"]:
                data[col] = [np.nan]
    # combine all the single values with any subplots from the dashboard
    df = pd.DataFrame(data)
    if not df.empty:
        df['Date'] = df['Date'].dt.normalize()  # Latest has time in it which creates double entries
        res = df.set_index("Date").combine_first(res)
    return res


In [ ]:
workbook_explore(workbook=workbook)

In [35]:
df = workbook.worksheets[9].data

In [48]:
df = df.iloc[:,[0,2,3]]

In [ ]:
df.pri